# Homework

Well done for progressing to this part of the mini-course, and engaging with the extra content! Below are some exercises about the Pandas package that will help you understand some more advanced concepts for working with `DataFrame`s of more complex data.

## Hierarchical indexing

We looked at the index before, but sometimes it makes sense to index data in more than one way. We will look at some examples below.

In [1]:
import pandas as pd

# Creating a DataFrame with hierarchical indexing (MultiIndex)
data = {
    "City": ["New York", "New York", "Los Angeles", "Los Angeles", "Chicago", "Chicago"],
    "Year": [2020, 2021, 2020, 2021, 2020, 2021],
    "Population": [8_336_817, 8_398_748, 3_979_576, 3_898_747, 2_693_976, 2_746_388]
}

df = pd.DataFrame(data)
df.set_index(["City", "Year"], inplace=True)  # Setting a MultiIndex

# Display the DataFrame
df

Population
City        Year            
New York    2020     8336817
            2021     8398748
Los Angeles 2020     3979576
            2021     3898747
Chicago     2020     2693976
            2021     2746388

In [2]:
# Accessing data for a specific city and year
df.loc["New York", 2020]

Population    8336817
Name: (New York, 2020), dtype: int64

In [3]:
# Accessing data for all years of a specific city
df.loc["Los Angeles"]

,Population
Year,
2020,3979576
2021,3898747


In [4]:
# Resetting the index to turn the MultiIndex back into columns
df_reset = df.reset_index()
df_reset

,City,Year,Population
0,New York,2020,8336817
1,New York,2021,8398748
2,Los Angeles,2020,3979576
3,Los Angeles,2021,3898747
4,Chicago,2020,2693976
5,Chicago,2021,2746388


In [5]:
# Adding another level to the index (e.g., Region)
df_reset["Region"] = ["East", "East", "West", "West", "Midwest", "Midwest"]
df_multi = df_reset.set_index(["Region", "City", "Year"])
df_multi

Population
Region  City        Year            
East    New York    2020     8336817
                    2021     8398748
West    Los Angeles 2020     3979576
                    2021     3898747
Midwest Chicago     2020     2693976
                    2021     2746388

In [6]:
# Sorting by the levels of the MultiIndex
df_multi_sorted = df_multi.sort_index(level=["Region", "City"])
df_multi_sorted

Population
Region  City        Year            
East    New York    2020     8336817
                    2021     8398748
Midwest Chicago     2020     2693976
                    2021     2746388
West    Los Angeles 2020     3979576
                    2021     3898747

In [7]:
# Using .xs() to access data across a specific level (e.g., all data for the year 2020)
df_multi.xs(2020, level="Year")

,,Population
Region,City,
East,New York,8336817
West,Los Angeles,3979576
Midwest,Chicago,2693976


### Exercises

1. Create a similar DataFrame for another dataset (e.g., sales data with Region, Product, and Year as indices). You could even take some rows from a real dataset! One example might be <a href="https://archive.ics.uci.edu/dataset/360/air+quality">the UC Irvine ML repository Air Quality dataset</a>, which has both a date and a time index


2. Practice accessing rows using `.loc[]` and `.xs()`.

3. Experiment with sorting by different levels of your MultiIndex.

4. Reset the index and re-create it with additional levels.


In [8]:
# Put your working here (Esc > 'b' to add more cells)

## Join/merging

This is when we take data from one table which is somehow relevant to the data in another table, and capture all of that information in a single table.

In [9]:
import pandas as pd # import again in case the last section hasn't been run

# Creating two DataFrames to merge
df1 = pd.DataFrame({
    "id": [1, 2, 3, 4],
    "Name": ["Alice", "Bob", "Charlie", "David"],
    "Age": [25, 30, 35, 40]
})

df2 = pd.DataFrame({
    "id": [3, 4, 5, 6],
    "City": ["Chicago", "Houston", "Los Angeles", "New York"],
    "Income": [70000, 80000, 90000, 100000]
})

print("DataFrame 1:")
print(df1)

print("\nDataFrame 2:")
print(df2)


DataFrame 1:
   id     Name  Age
0   1    Alice   25
1   2      Bob   30
2   3  Charlie   35
3   4    David   40

DataFrame 2:
   id         City  Income
0   3      Chicago   70000
1   4      Houston   80000
2   5  Los Angeles   90000
3   6     New York  100000


In [10]:
# Inner join: Keeps only rows with matching 'id' in both DataFrames
df_inner = pd.merge(df1, df2, on="id", how="inner")

print("Inner Join Result:")
print(df_inner)

Inner Join Result:
   id     Name  Age     City  Income
0   3  Charlie   35  Chicago   70000
1   4    David   40  Houston   80000


In [11]:
# 'Left' join: Keeps all rows from df1 and matches from df2 where possible
df_left = pd.merge(df1, df2, on="id", how="left")

print("Left Join Result:")
print(df_left)

Left Join Result:
   id     Name  Age     City   Income
0   1    Alice   25      NaN      NaN
1   2      Bob   30      NaN      NaN
2   3  Charlie   35  Chicago  70000.0
3   4    David   40  Houston  80000.0


In [12]:
# Right join: Keeps all rows from df2 and matches from df1 where possible
df_right = pd.merge(df1, df2, on="id", how="right")

print("Right Join Result:")
print(df_right)

Right Join Result:
   id     Name   Age         City  Income
0   3  Charlie  35.0      Chicago   70000
1   4    David  40.0      Houston   80000
2   5      NaN   NaN  Los Angeles   90000
3   6      NaN   NaN     New York  100000


In [13]:
# Outer join: Keeps all rows from both DataFrames with NaNs where no match exists
df_outer = pd.merge(df1, df2, on="id", how="outer")

print("Outer Join Result:")
print(df_outer)

Outer Join Result:
   id     Name   Age         City    Income
0   1    Alice  25.0          NaN       NaN
1   2      Bob  30.0          NaN       NaN
2   3  Charlie  35.0      Chicago   70000.0
3   4    David  40.0      Houston   80000.0
4   5      NaN   NaN  Los Angeles   90000.0
5   6      NaN   NaN     New York  100000.0


In [14]:
# Customizing column suffixes for overlapping column names
df_suffix = pd.merge(df1.rename(columns={"Age": "Feature"}), df2.rename(columns={"Income": "Feature"}), 
                     on="id", how="outer", suffixes=("_left", "_right"))

print("Outer Join with Custom Suffixes:")
print(df_suffix)

Outer Join with Custom Suffixes:
   id     Name  Feature_left         City  Feature_right
0   1    Alice          25.0          NaN            NaN
1   2      Bob          30.0          NaN            NaN
2   3  Charlie          35.0      Chicago        70000.0
3   4    David          40.0      Houston        80000.0
4   5      NaN           NaN  Los Angeles        90000.0
5   6      NaN           NaN     New York       100000.0


In [15]:
# Example of merging on different column names using left_on and right_on
df3 = pd.DataFrame({
    "PersonID": [1, 2],
    "Hobby": ["Reading", "Cycling"]
})

df_merge_diff_cols = pd.merge(df1, df3, left_on="id", right_on="PersonID")

print("Merge on Different Columns:")
print(df_merge_diff_cols)

Merge on Different Columns:
   id   Name  Age  PersonID    Hobby
0   1  Alice   25         1  Reading
1   2    Bob   30         2  Cycling


### Exercise

Play around with the data provided, add and remove columns and rows, see what breaks this and what you can do with it beyond what was immediately provided! Continue changing things and predicting the result of changing them until you are confident with the idea of what "merging" data is

In [16]:
# Put your working here (Esc > 'b' to add more cells)

## GroupBy

This is when we aggregate the data in a table into groups in a particular way, and usually then compute some statistics for each group.

In [17]:
import pandas as pd # import again in case the last section hasn't been run

# Create a sample DataFrame
data = {
    "Category": ["Electronics", "Clothing", "Electronics", "Clothing", "Electronics"],
    "Sales": [1000, 500, 800, 300, 1200],
    "Region": ["East", "West", "East", "West", "East"]
}

df = pd.DataFrame(data)

print("Sample DataFrame:")
print(df)


Sample DataFrame:
      Category  Sales Region
0  Electronics   1000   East
1     Clothing    500   West
2  Electronics    800   East
3     Clothing    300   West
4  Electronics   1200   East


In [18]:
# Group by 'Category' and calculate the total sales for each category
grouped_single = df.groupby("Category")["Sales"].sum()

print("Total Sales by Category:")
print(grouped_single)

Total Sales by Category:
Category
Clothing        800
Electronics    3000
Name: Sales, dtype: int64


In [19]:
# Group by 'Category' and 'Region', and calculate the total sales for each group
grouped_multiple = df.groupby(["Category", "Region"])["Sales"].sum()

print("Total Sales by Category and Region:")
print(grouped_multiple)

Total Sales by Category and Region:
Category     Region
Clothing     West       800
Electronics  East      3000
Name: Sales, dtype: int64


In [20]:
# Apply multiple aggregate functions (e.g., sum and mean) to 'Sales'
aggregated = df.groupby("Category")["Sales"].agg(["sum", "mean"])

print("Sum and Mean of Sales by Category:")
print(aggregated)

Sum and Mean of Sales by Category:
              sum    mean
Category                 
Clothing      800   400.0
Electronics  3000  1000.0


In [21]:
# Access the group corresponding to the 'Electronics' category
electronics_group = df.groupby("Category").get_group("Electronics")

print("Data for Electronics Category:")
print(electronics_group)

Data for Electronics Category:
      Category  Sales Region
0  Electronics   1000   East
2  Electronics    800   East
4  Electronics   1200   East


In [22]:
# Iterate through groups and print group names and their contents
grouped = df.groupby("Category")

print("Iterating through groups:")
for group_name, group_data in grouped:
    print(f"\nGroup: {group_name}")
    print(group_data)

Iterating through groups:

Group: Clothing
   Category  Sales Region
1  Clothing    500   West
3  Clothing    300   West

Group: Electronics
      Category  Sales Region
0  Electronics   1000   East
2  Electronics    800   East
4  Electronics   1200   East


In [23]:
# Calculate the percentage of sales within each category
df["Category_Percentage"] = df.groupby("Category")["Sales"].transform(lambda x: x / x.sum() * 100) # lambdas are like on-the-fly functions

print("DataFrame with Category Percentage:")
print(df)

DataFrame with Category Percentage:
      Category  Sales Region  Category_Percentage
0  Electronics   1000   East            33.333333
1     Clothing    500   West            62.500000
2  Electronics    800   East            26.666667
3     Clothing    300   West            37.500000
4  Electronics   1200   East            40.000000


In [24]:
# Equivalently to above:
def percentage(x):
    return x / x.sum() * 100

df["Category_Percentage"] = df.groupby("Category")["Sales"].transform(percentage)

print("DataFrame with Category Percentage:")
print(df)

DataFrame with Category Percentage:
      Category  Sales Region  Category_Percentage
0  Electronics   1000   East            33.333333
1     Clothing    500   West            62.500000
2  Electronics    800   East            26.666667
3     Clothing    300   West            37.500000
4  Electronics   1200   East            40.000000


In [25]:
# Define a custom function to calculate the range (max - min) of sales within each category
def sales_range(group):
    return group["Sales"].max() - group["Sales"].min()

sales_range_by_category = df.groupby("Category")[df.columns].apply(sales_range)

print("Range of Sales by Category:")
print(sales_range_by_category)

Range of Sales by Category:
Category
Clothing       200
Electronics    400
dtype: int64


### Exercise

Once again, just play with the data in the above examples, or translate them into your own examples, until you feel you have a grasp of the concepts involved.

Ask yourself questions like "How can I find the mean salary of staff members in a certain age range, in a certain department?", then make or find some appropriate data, imagine a solution, and try it out!

When it doesn't work (which it almost never does, the first time) look up the error, and look up how to do what you're trying to do in the most generalised way you can think of. For that example a search term might look like "mean of sub-group pandas" and you might find something like <a href="https://stackoverflow.com/questions/49245451/how-to-calculate-average-on-a-subset-of-a-subset-of-a-dataframe-in-python">this StackOverflow question</a>, which can help you! 

StackOverflow in particular is your friend when looking up things about Python

In [26]:
# Put your working here (Esc > 'b' to add more cells)

## String `Series` and time `Series`

Pandas provides special methods for working with string data and time data, since they are not strictly numerical. This section will look at some of what you can do with these types of data in Pandas.

### Strings

In [27]:
import pandas as pd # import again in case the last section hasn't been run

# Create a sample DataFrame with textual data
data = {
    "Name": ["Alice Smith", "Bob Johnson", "Charlie Brown", "David Wilson"],
    "City": ["New York", "Los Angeles", "Chicago", "Houston"]
}

df = pd.DataFrame(data)

print("Sample DataFrame:")
print(df)


Sample DataFrame:
            Name         City
0    Alice Smith     New York
1    Bob Johnson  Los Angeles
2  Charlie Brown      Chicago
3   David Wilson      Houston


In [28]:
# Convert all names to uppercase
df["Name_Upper"] = df["Name"].str.upper()

# Extract the first name (text before the space)
df["First_Name"] = df["Name"].str.split(" ").str[0]

print("DataFrame with String Manipulations:")
print(df)

DataFrame with String Manipulations:
            Name         City     Name_Upper First_Name
0    Alice Smith     New York    ALICE SMITH      Alice
1    Bob Johnson  Los Angeles    BOB JOHNSON        Bob
2  Charlie Brown      Chicago  CHARLIE BROWN    Charlie
3   David Wilson      Houston   DAVID WILSON      David


In [29]:
# Check if the city name starts with 'C'
df["City_Starts_C"] = df["City"].str.startswith("C")

print("DataFrame with Conditional String Operations:")
print(df)

DataFrame with Conditional String Operations:
            Name         City     Name_Upper First_Name  City_Starts_C
0    Alice Smith     New York    ALICE SMITH      Alice          False
1    Bob Johnson  Los Angeles    BOB JOHNSON        Bob          False
2  Charlie Brown      Chicago  CHARLIE BROWN    Charlie           True
3   David Wilson      Houston   DAVID WILSON      David          False


In [30]:
# Replace 'New York' with 'NYC'
df["City_Replaced"] = df["City"].str.replace("New York", "NYC")

print("DataFrame with Text Replacement:")
print(df)

DataFrame with Text Replacement:
            Name         City     Name_Upper First_Name  City_Starts_C  \
0    Alice Smith     New York    ALICE SMITH      Alice          False   
1    Bob Johnson  Los Angeles    BOB JOHNSON        Bob          False   
2  Charlie Brown      Chicago  CHARLIE BROWN    Charlie           True   
3   David Wilson      Houston   DAVID WILSON      David          False   

  City_Replaced  
0           NYC  
1   Los Angeles  
2       Chicago  
3       Houston  


### Time

In [31]:
# Create a sample DataFrame with datetime data
date_data = {
    "Event": ["Start", "Middle", "End"],
    "Date": pd.to_datetime(["2025-03-01", "2025-03-14", "2025-03-31"])
}

df_date = pd.DataFrame(date_data)

print("Sample DateTime DataFrame:")
print(df_date)

Sample DateTime DataFrame:
    Event       Date
0   Start 2025-03-01
1  Middle 2025-03-14
2     End 2025-03-31


In [32]:
# Extract year, month, and day from the Date column
df_date["Year"] = df_date["Date"].dt.year
df_date["Month"] = df_date["Date"].dt.month
df_date["Day"] = df_date["Date"].dt.day
# dt stands for 'datetime'
print("DataFrame with Extracted Date Components:")
print(df_date)

DataFrame with Extracted Date Components:
    Event       Date  Year  Month  Day
0   Start 2025-03-01  2025      3    1
1  Middle 2025-03-14  2025      3   14
2     End 2025-03-31  2025      3   31


In [33]:
# Create a new column with datetime including time for demonstration
df_date["DateTime"] = pd.to_datetime(["2025-03-01 08:00:00", "2025-03-14 12:30:00", "2025-03-31 18:45:00"])

# Extract hour and minute from the DateTime column
df_date["Hour"] = df_date["DateTime"].dt.hour
df_date["Minute"] = df_date["DateTime"].dt.minute

print("DataFrame with Extracted Time Components:")
print(df_date)

DataFrame with Extracted Time Components:
    Event       Date  Year  Month  Day            DateTime  Hour  Minute
0   Start 2025-03-01  2025      3    1 2025-03-01 08:00:00     8       0
1  Middle 2025-03-14  2025      3   14 2025-03-14 12:30:00    12      30
2     End 2025-03-31  2025      3   31 2025-03-31 18:45:00    18      45


In [34]:
# Filter rows where the date is after March 10, 2025
filtered_df = df_date[df_date["Date"] > pd.Timestamp("2025-03-10")]

print("Filtered DataFrame (Dates after March 10, 2025):")
print(filtered_df)

Filtered DataFrame (Dates after March 10, 2025):
    Event       Date  Year  Month  Day            DateTime  Hour  Minute
1  Middle 2025-03-14  2025      3   14 2025-03-14 12:30:00    12      30
2     End 2025-03-31  2025      3   31 2025-03-31 18:45:00    18      45


### Exercise

Once again, make sure you understand what is happening in the above examples by changing them or creating your own examples. If you were given a table of text from social media posts along with the date and time, could you find the hour of the day, or the day of the month, in which the posts posts made contain the greatest proportion of profanity/swear words (given also a list of swear words, of course)?

You have now been given all the tools to do so, so try to imagine how, and test it on a small example! Mo need to use real bad language of course - 'cabbage' is a good replacement. As always, use the internet for help if it gets difficult or you get confused.

In [35]:
# Put your working here (Esc > 'b' to add more cells)